In [2]:
!pip install wbdata
!pip install cufflinks
!pip install iso3166

import iso3166 #iso3166.countries.get('country details')
import wbdata
import cufflinks as cf
import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt
import seaborn as sns
cf.go_offline()

  Using cached wbdata-0.3.0-py3-none-any.whl (14 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.6.2 requires h5py~=3.1.0, but you'll have h5py 3.3.0 which is incompatible.
tensorflow 2.6.2 requires numpy~=1.19.2, but you'll have numpy 1.21.5 which is incompatible.
tensorflow 2.6.2 requires six~=1.15.0, but you'll have six 1.16.0 which is incompatible.
tensorflow 2.6.2 requires typing-extensions~=3.7.4, but you'll have typing-extensions 4.1.1 which is incompatible.
fancyimpute 0.6.0 requires keras==2.4.3, but you'll have keras 2

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning:

The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



In [4]:
import wbdata

# Return list of all country/region codes:
wbdata.get_country()

# Return list matching a query term:
wbdata.search_countries("Syrian Arab Republic")

## Try your own search!
# wbdata.search_countries("")

id    name
----  --------------------
SYR   Syrian Arab Republic

In [36]:
import iso3166 #iso3166.countries.get('country details')
location = 'Syrian Arab Republic'
def get_dataframe(year, group, age_lower, age_upper, location):
    country_code = "WLD"
    og_l = age_lower
    og_u = age_upper
    if (age_lower % 5) != 0:
        age_lower = 5 * round(age_lower/5)
    if (age_upper % 5) != 0:
        age_upper = 5 * round(age_upper/5)   
    if location != 'world':
        country_code = iso3166.countries.get(location).alpha3
    age_ranges = []
    for i in range(age_lower, age_upper, 5):
        age_ranges.append(f"{i:02d}"+f"{i+4:02d}")
    
    female_variables = {"SP.POP."+age_range+".FE":"Females "+age_range for age_range in age_ranges}
    female_df = wbdata.get_dataframe(female_variables,country=country_code)
    female_data = female_df.query("date=='{}'".format(year)).sum(axis=0).tolist()
    
    male_variables = {"SP.POP."+age_range+".MA":"Males "+age_range for age_range in age_ranges}
    male_df = wbdata.get_dataframe(male_variables,country=country_code)
    male_data = male_df.query("date=='{}'".format(year)).sum(axis=0).tolist()
    
    age_input = [i[:2]+'-'+i[2:] for i in age_ranges]
    
    df = pd.DataFrame({
        'Country': location,
        'Year': year,
        'Age': age_input,
        'Female': female_data,
        'Male': male_data,
    })
    
    if age_lower != og_l:
        true_sum_male = df["Male"].sum() - df["Male"][(age_lower/5)-2]/(abs(age_lower-og_l))
        true_sum_female = df["Female"].sum()  - df["Female"][(age_lower/5)-2]/(abs(age_lower-og_l))
    else: 
        true_sum_male = df["Male"].sum() 
        true_sum_female = df["Female"].sum() 
    
    if age_upper != og_u:
        true_sum_male = true_sum_male -  df["Male"][(age_lower/5)-2]/(abs(age_lower-og_l))
        true_sum_female = true_sum_female - df["Female"][(age_lower/5)-2]/(abs(age_lower-og_l))
    else: 
        true_sum_male = df["Male"].sum()
        true_sum_female = df["Male"].sum()
    all_sum = true_sum_female + true_sum_male
    
    if group == "Female":
        return true_sum_female
    elif group =="Male":
        return true_sum_male
    else: 
        return all_sum

get_dataframe(2010, 'all', 14, 34, "SYR")

3656310.0